# Layer Analytics of Neural Networks


In [106]:
import os, sys, math, datetime
import psutil
#import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image

# Import the necessary MLTK APIs
from mltk.core import view_model, summarize_model, profile_model
from mltk.utils.commander import query_platform

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping



In [107]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [108]:
models_dir = Path.cwd().joinpath("models")
if not models_dir.exists():
    print(f"{models_dir} does not exist.")
    models_dir.mkdir()
    print(f"Created path: {models_dir}.")


In [109]:
input_shape =(96,96,3)
classes = 2
alpha = 1 #0.25

# generate folder structure for saving the models
channels = input_shape[-1]
print(channels)


3


# Helper Functions

In [110]:
# Helper functions

def flat_size(shape : list):
    out = 1
    for k in shape:
        out *= k
        
    return out

# Create the model

In [111]:
mobilenet = tf.keras.applications.mobilenet.MobileNet(
    input_shape=input_shape,
    alpha=alpha,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights=None, #'imagenet'
    input_tensor=None,
    pooling=None,
    classes=classes,
    classifier_activation='softmax',
    #**kwargs
)

In [112]:
model = mobilenet
model.compile(optimizer='adam',
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])

In [113]:
summary = model.summary(expand_nested=True)

Model: "mobilenet_1.00_96"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 48, 48, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 48, 48, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 48, 48, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 48, 48, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 48, 48, 32)       128       
 ation)                                          

In [114]:
mltk_summary = summarize_model(model)
print(mltk_summary)

Model: "mobilenet_1.00_96"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 48, 48, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 48, 48, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 48, 48, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 48, 48, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 48, 48, 32)       128       
 ation)                                          

In [115]:
# MLTK profile model reads the mode from a path

#profiling_results = profile_model(model, accelerator='MVP', build=True)

In [116]:
# Show model in local version of Netron.app
#view_model(model, tflite= True, build=True)

In [117]:
model_name = model.name
model_name = f"{model_name}_c{channels}"
print(model_name)


mobilenet_1.00_96_c3


In [118]:
models_path = models_dir.joinpath(model_name)
if not models_path.exists():
    print(f"{models_path} does not exist.")
    models_path.mkdir()
    print(f"Created path: {models_path}.")

models_summary_path = models_dir.joinpath(model_name, f"{model_name}.txt")
models_image_path = models_dir.joinpath(model_name, f"{model_name}.png")
print(models_summary_path)
models_tflite_path = models_dir.joinpath(model_name, f"{model_name}.tflite")
models_tflite_opt_path = models_dir.joinpath(model_name, f"{model_name}_INT8.tflite")
print(models_tflite_path)

i:\tinyml\tiny_cnn\models\mobilenet_1.00_96_c3\mobilenet_1.00_96_c3.txt
i:\tinyml\tiny_cnn\models\mobilenet_1.00_96_c3\mobilenet_1.00_96_c3.tflite


# Get details from the different layers of the model

In [119]:
#toy_model = tf.m

In [120]:
# tf.keras.utils.plot_model(model, 
#     to_file=models_image_path,
#     show_shapes=True, 
#     show_dtype=False,
#     show_layer_names=True, 
#     rankdir='TB', # TB for vertical plot, LR for horizontal plot
#     expand_nested=True, 
#     layer_range=None,
#     dpi=200,
#     show_layer_activations= True)

# Layer Metrics Helper Functions

In [121]:
def Conv2D_MACs(layer):
    """
    Code adapted from https://github.com/SiliconLabs/mltk/blob/master/mltk/core/model/metrics/layers/conv.py
    """

    if isinstance(layer, keras.layers.Conv2D):
        _, _, _, in_depth = layer.input_shape # layer.input_data.shape
        _, out_h, out_w, out_depth = layer.output_shape # layer.output_data.shape
    else:
        if layer.data_format == "channels_first":
            _, in_depth, _, _ = layer.input_shape
            _, out_depth, out_h, out_w, = layer.output_shape
        elif layer.data_format == "channels_last":
            _, _, _, in_depth = layer.input_shape
            _, out_h, out_w, out_depth = layer.output_shape

    w_h, w_w =  layer.kernel_size
    name = layer.name
    macs = w_h * w_w * in_depth * out_w * out_h * out_depth
    ops = macs * 2 

    if  layer.use_bias:
        ops += out_w * out_h * out_depth
    
    return (name, macs, ops)

In [122]:
def DepthwiseConv2D_MACs(layer):
    """
    Code adapted from https://github.com/SiliconLabs/mltk/blob/master/mltk/core/model/metrics/layers/depthwise_conv.py
    """

    # if isinstance(layer, TfliteDepthwiseConv2dLayer):
    #     _, _, _, in_depth = layer.input_data.shape
    #     _, out_h, out_w, out_depth = layer.output_data.shape

    # else: keras.layers.DepthwiseConv2D 
    if isinstance(layer, keras.layers.DepthwiseConv2D):
        if layer.data_format == "channels_first":
            _, in_depth, _, _ = layer.input_shape
            _, out_depth, out_h, out_w, = layer.output_shape
        elif layer.data_format == "channels_last":
            _, _, _, in_depth = layer.input_shape
            _, out_h, out_w, out_depth = layer.output_shape

    w_h, w_w =  layer.kernel_size
    depth_multiplier = out_depth // in_depth
    
    name = layer.name
    macs = w_h * w_w * depth_multiplier * in_depth * out_w * out_h
    ops = macs * 2 
    
    if layer.use_bias:
        ops += depth_multiplier * in_depth * out_w * out_h

    return (name, macs, ops)

In [123]:
def SeparableConv2D_MACs(layer):
    """
    Code adapted from https://github.com/SiliconLabs/mltk/blob/master/mltk/core/model/metrics/layers/depthwise_conv.py
    """
    name = layer.name
    if isinstance(layer, keras.layers.SeparableConv2D):
        if layer.data_format == "channels_first":
            _, in_depth, _, _ = layer.input_shape
            _, out_depth, out_h, out_w, = layer.output_shape
        elif layer.data_format == "channels_last":
            _, _, _, in_depth = layer.input_shape
            _, out_h, out_w, out_depth = layer.output_shape

    else:
        return (name, "mac_error", "ops_error") 

    w_h, w_w =  layer.kernel_size
   
    macs_depth = w_h * w_w * in_depth * out_w * out_h 
    macs_point = in_depth * out_w * out_h * out_depth
    macs = macs_depth + macs_point
    ops = macs * 2 

    if  layer.use_bias:
        ops += in_depth * out_w * out_h # for depthwise convolution
        ops += out_w * out_h * out_depth  # for pointwise convolution ## TODO: check the calculations
        

    return (name, macs, ops)

In [124]:
def Dense_MACs(layer):
    """
    Code adapted from https://github.com/SiliconLabs/mltk/blob/master/mltk/core/model/metrics/layers/dense.py
    """

    if isinstance(layer, keras.layers.Dense):
        bias_ops = 0
        # if isinstance(layer, TfliteFullyConnectedLayer):
        #     in_depth = layer.input_data.shape[-1]
        #     out_depth = layer.output_data.shape[-1]
        #     if layer.use_bias:
        #         bias_ops = layer.outputs[0].shape.flat_size
        # else:
        in_depth = layer.input_shape[-1]
        out_depth = layer.output_shape[-1]
        if layer.use_bias:
            bias_ops = flat_size(layer.output_shape[1:])
            
        name = layer.name
        macs = in_depth * out_depth
        ops = macs * 2 + bias_ops
    
    return (name, macs, ops)

In [125]:
def AveragePooling2D_MACs(layer):
    """
    Code adapted from https://github.com/SiliconLabs/mltk/blob/master/mltk/core/model/metrics/layers/pooling.py
    """

    if isinstance(layer, keras.layers.AveragePooling2D):
        f_h, f_w  = layer.pool_size
        name = layer.name
        # if isinstance(layer, KerasLayer):
        output_shape = layer.output_shape
        # else:
        #     output_shape = layer.outputs[0].shape

        ops = flat_size(output_shape[1:]) * (f_h * f_w + 1) # + 1 for the division
    
    return (name, ops)

In [126]:
def calculate_no_activations(layer):
    if isinstance(layer, (keras.layers.Conv2D, keras.layers.Dense, keras.layers.Input)):
    #if isinstance(layer, (keras.layers)):
        if layer.data_format == "channels_first":
                _, out_depth, out_h, out_w, = layer.output_shape
        elif layer.data_format == "channels_last":
            _, out_h, out_w, out_depth = layer.output_shape

        return out_h * out_w * out_depth

    else:
        return 0

In [127]:
def print_layer_config(layer, layer_type):
    if isinstance(layer, layer_type):
        print(f"  Layer: {type(layer)}")
        print(layer.get_config())
        print()
    else:
        pass
    return layer.layer_config

In [128]:

layer_list = []
for i, layer in enumerate(model.layers):
    layer_stats = {}
    layer_stats["index"] = i
    layer_stats["name"] = layer.name
    layer_stats["layer_type"] = type(layer) # TODO: find a better representation
    layer_stats["input_shape"] = layer.input_shape
    layer_stats["output_shape"] = layer.output_shape
    layer_stats["dtype"] = layer.compute_dtype
    layer_stats["params"] = layer.count_params()
    layer_stats["variables"] = len(layer.variables)
    try:
        layer_stats["weights_shape"] = layer.weights[0].shape
        layer_stats["weights_name"] = layer.weights[0].name
        layer_stats["weights_dtype"] = layer.weights[0].dtype
        layer_stats["#_activations"] = calculate_no_activations(layer)
    except:
        layer_stats["weights_shape"] = 0
        layer_stats["weights_name"] = 0
        layer_stats["weights_dtype"] = 0  
        layer_stats["#_activations"] = 0  

    layer_config = layer.get_config()

    # Set default values for parameters that are not present in each layer
    layer_stats["OPS"] = 0
    layer_stats["MACS"] = 0
    layer_stats["filters"]= 0
    layer_stats["trainable"] = 0
    layer_stats["kernel_size"] = 0
    layer_stats["strides"] = 0
    layer_stats["padding"] = 0
    layer_stats["dilation_rate"] = 0
    layer_stats["groups"] = 0
    layer_stats["activation"] = 0
    layer_stats["use_bias"] = 0
    layer_stats["data_format"] = 0

    # if isinstance(layer, keras.layers.InputLayer):
    #     print(f"  Layer: {type(layer)}")
    #     print(layer_config)
    #     print()

    if isinstance(layer, keras.layers.Conv2D ):
        # print(f"  Layer: {type(layer)}")
        # print(layer_config)
        # print()
        name, macs, ops = Conv2D_MACs(layer)


        layer_stats["OPS"] = ops
        layer_stats["MACS"] = macs
        #layer_stats["# activation"] = layer.output_shape[1]
        layer_stats["filters"]= layer_config["filters"]
        layer_stats["trainable"] = layer_config["trainable"]
        layer_stats["kernel_size"] = layer_config["kernel_size"]
        layer_stats["strides"] = layer_config["strides"]
        layer_stats["padding"] = layer_config["padding"]
        layer_stats["dilation_rate"] = layer_config["dilation_rate"]
        layer_stats["groups"] = layer_config["groups"]
        layer_stats["activation"] = layer_config["activation"]
        layer_stats["use_bias"] = layer_config["use_bias"]
        layer_stats["data_format"] = layer_config["data_format"]


    if isinstance(layer, keras.layers.DepthwiseConv2D ):
        # print(f"  Layer: {type(layer)}")
        # print(layer_config)
        # print()

        name, macs, ops = DepthwiseConv2D_MACs(layer)  

        #layer_stats["filters"]= layer_config["filters"]  # TODO: key error -> refactor with Conv2D
        layer_stats["OPS"] = ops
        layer_stats["MACS"] = macs
        layer_stats["trainable"] = layer_config["trainable"]
        layer_stats["kernel_size"] = layer_config["kernel_size"]
        layer_stats["strides"] = layer_config["strides"]
        layer_stats["padding"] = layer_config["padding"]
        layer_stats["dilation_rate"] = layer_config["dilation_rate"]
        layer_stats["groups"] = layer_config["groups"]
        layer_stats["activation"] = layer_config["activation"]
        layer_stats["use_bias"] = layer_config["use_bias"]
        layer_stats["data_format"] = layer_config["data_format"]


    # if isinstance(layer, keras.layers.Dense):
    #     print(f"  Layer: {type(layer)}")
    #     print(layer_config)
    #     print()

    # if isinstance(layer, keras.layers.BatchNormalization ):
    #     print(f"  Layer: {type(layer)}")
    #     print(layer_config)
    #     print()
    layer_list.append(layer_stats)

layer_stats_df = pd.DataFrame.from_dict(layer_list)
layer_stats_df

,index,name,layer_type,input_shape,output_shape,dtype,params,variables,weights_shape,weights_name,weights_dtype,#_activations,OPS,MACS,filters,trainable,kernel_size,strides,padding,dilation_rate,groups,activation,use_bias,data_format
0,0,input_3,<class 'keras.engine.input_layer.InputLayer'>,"[(None, 96, 96, 3)]","[(None, 96, 96, 3)]",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,conv1,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 96, 96, 3)","(None, 48, 48, 32)",float32,864,1,"(3, 3, 3, 32)",conv1/kernel:0,<dtype: 'float32'>,73728,3981312,1990656,32,True,"(3, 3)","(2, 2)",same,"(1, 1)",1,linear,False,channels_last
2,2,conv1_bn,<class 'keras.layers.normalization.batch_norma...,"(None, 48, 48, 32)","(None, 48, 48, 32)",float32,128,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,conv1_relu,<class 'keras.layers.activation.relu.ReLU'>,"(None, 48, 48, 32)","(None, 48, 48, 32)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,conv_dw_1,<class 'keras.layers.convolutional.depthwise_c...,"(None, 48, 48, 32)","(None, 48, 48, 32)",float32,288,1,0,0,0,0,1327104,663552,0,True,"(3, 3)","(1, 1)",same,"(1, 1)",1,linear,False,channels_last
5,5,conv_dw_1_bn,<class 'keras.layers.normalization.batch_norma...,"(None, 48, 48, 32)","(None, 48, 48, 32)",float32,128,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,6,conv_dw_1_relu,<class 'keras.layers.activation.relu.ReLU'>,"(None, 48, 48, 32)","(None, 48, 48, 32)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,7,conv_pw_1,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 48, 48, 32)","(None, 48, 48, 64)",float32,2048,1,"(1, 1, 32, 64)",conv_pw_1/kernel:0,<dtype: 'float32'>,147456,9437184,4718592,64,True,"(1, 1)","(1, 1)",same,"(1, 1)",1,linear,False,channels_last
8,8,conv_pw_1_bn,<class 'keras.layers.normalization.batch_norma...,"(None, 48, 48, 64)","(None, 48, 48, 64)",float32,256,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,9,conv_pw_1_relu,<class 'keras.layers.activation.relu.ReLU'>,"(None, 48, 48, 64)","(None, 48, 48, 64)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [129]:
#layer_stats_df[layer_stats_df["OPS"]=="ops_error"]

In [130]:
layer_stats_df["layer_type"].unique()

array([<class 'keras.engine.input_layer.InputLayer'>,
       <class 'keras.layers.convolutional.conv2d.Conv2D'>,
       <class 'keras.layers.normalization.batch_normalization.BatchNormalization'>,
       <class 'keras.layers.activation.relu.ReLU'>,
       <class 'keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D'>,
       <class 'keras.layers.reshaping.zero_padding2d.ZeroPadding2D'>,
       <class 'keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D'>,
       <class 'keras.layers.regularization.dropout.Dropout'>,
       <class 'keras.layers.reshaping.reshape.Reshape'>,
       <class 'keras.layers.core.activation.Activation'>], dtype=object)

In [131]:
# name, macs, ops = Conv2D_Metrics(model.layers[1])
# print(name, macs, ops)